In [1]:
from pathlib import Path
import pandas as pd
import geopandas as gpd

import sys
sys.path.append('../')

from veg2hab.waswordtlijst import WasWordtLijst, opschonen_was_wordt_lijst
from veg2hab.definitietabel import DefinitieTabel, opschonen_definitietabel
from veg2hab.vegkartering import Kartering
from veg2hab.fgr import FGR

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
path_in_wwl = Path("../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx")
path_out_wwl = Path("../testing/opgeschoonde_waswordt.xlsx")
opschonen_was_wordt_lijst(path_in_wwl, path_out_wwl)
wwl = WasWordtLijst.from_excel(path_out_wwl)

path_in_dt = Path("../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls")
path_in_json_def = Path("../data/definitietabel_json.csv")
path_out_dt = Path("../testing/opgeschoonde_definitietabel.xlsx")
opschonen_definitietabel(path_in_dt, path_in_json_def, path_out_dt)
dt = DefinitieTabel.from_excel(path_out_dt)

fgr = FGR(Path("../data/bronbestanden/FGR.json"))

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/pandas/core/missing.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x
/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/pandas/core/missing.py:60: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= arr == x


Alle SBB codes zijn valide
Alle VvN codes zijn valide
Alle SBB codes zijn valide
Alle VvN codes zijn valide


/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/pandas/core/missing.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x
/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/pandas/core/missing.py:60: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= arr == x


Alle SBB codes zijn valide
Alle VvN codes zijn valide


### Inladen benodigde informatie
En het selecteren van access karteringen in Groningen

In [3]:
# info over de karteringen
overzicht = pd.read_excel(
    "../testing/vegetatiekarteringen/Overzicht_vegetatiekarteringen.xlsx", 
    sheet_name="Vegetatiekarteringen", 
    engine='openpyxl',
    usecols=[
        "provincie", 
        "naam_kartering", 
        "path_shapes", 
        "landelijke_typologie beschikbaar?", 
        "locatie_landelijke_typologie",
    ],
    dtype="string"
)

# benodigde info voor de access karteringen
access_info = pd.read_excel(
    "../testing/accessinfo.xlsx",
    engine='openpyxl',
    usecols=[
        "naam_kartering",
        "path_csvs",
        "elm_id_name",
    ],
    dtype="string"
)

# Voor nu even enkel access karteringen uit groningen
overzicht = overzicht[overzicht.provincie == "Groningen"]
overzicht = overzicht[overzicht["landelijke_typologie beschikbaar?"] == "Ja"]
overzicht = overzicht[overzicht["locatie_landelijke_typologie"] == "access"]
overzicht = overzicht.join(access_info.set_index("naam_kartering"), on="naam_kartering")

overzicht.path_shapes = overzicht.path_shapes.apply(lambda x: "../testing/vegetatiekarteringen" + x[1:])
overzicht.path_csvs = overzicht.path_csvs.apply(lambda x: "../testing/vegetatiekarteringen" + x[1:])

overzicht

,provincie,naam_kartering,path_shapes,landelijke_typologie beschikbaar?,locatie_landelijke_typologie,path_csvs,elm_id_name
70,Groningen,NM vegetatiekartering RuitenAa2020,../testing/vegetatiekarteringen/GR/NM vegetati...,Ja,access,../testing/vegetatiekarteringen/GR/NM vegetati...,Elmid
71,Groningen,SBB Duurswold2013,../testing/vegetatiekarteringen/GR/SBB Duurswo...,Ja,access,../testing/vegetatiekarteringen/GR/SBB Duurswo...,Elmid
72,Groningen,SBB Lauwersmeer 2015,../testing/vegetatiekarteringen/GR/SBB Lauwers...,Ja,access,../testing/vegetatiekarteringen/GR/SBB Lauwers...,ElmID
73,Groningen,SBB Westerwolde 2020,../testing/vegetatiekarteringen/GR/SBB Westerw...,Ja,access,../testing/vegetatiekarteringen/GR/SBB Westerw...,ELMID
75,Groningen,SBB ZWK 2010 0815_Trimunt_2010,../testing/vegetatiekarteringen/GR/SBB ZWK 201...,Ja,access,../testing/vegetatiekarteringen/GR/SBB ZWK 201...,Elmid
79,Groningen,SGL Zuidlaardermeer 2019,../testing/vegetatiekarteringen/GR/SGL Zuidlaa...,Ja,access,../testing/vegetatiekarteringen/GR/SGL Zuidlaa...,ELMID


### Karteringen inladen

In [4]:
karteringen = {}
for row in overzicht.itertuples():
    karteringen[row.naam_kartering] = Kartering.from_access_db(Path(row.path_shapes), row.elm_id_name, Path(row.path_csvs))

Er zijn 7 vlakken zonder VegTypeInfo. Deze worden gedropt.
De eerste paar ElmID van de gedropte vlakken zijn: [6099, 8174, 8174, 11051, 11023]
Er zijn 90 vlakken zonder VegTypeInfo. Deze worden gedropt.
De eerste paar ElmID van de gedropte vlakken zijn: [78, 76, 69, 82, 81]


### Karteringen omzetten

In [5]:
for kartering in karteringen.values():
    kartering.apply_wwl(wwl)

In [6]:
for kartering in karteringen.values():
    kartering.apply_deftabel(dt)

In [7]:
for kartering in karteringen.values():
    kartering.check_mitsen(fgr)

### Karteringen wegschrijven

In [8]:
for gebied in karteringen.keys():
    p = Path(f"../testing/omgezette_vegetatiekarteringen/GR/{gebied}.shp")
    karteringen[gebied].final_format_to_file(p)